<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/Sntence_entailment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**work in progress**

# Data Exploration

In [4]:
import numpy as np
import pandas as pd

In [2]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
!unzip train.zip

In [5]:
train = pd.read_csv("train.csv")

In [6]:
train_filtered  = train.filter(['premise','hypothesis','label'])

In [7]:
train_filtered.head()

,premise,hypothesis,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,1


In [ ]:
#get cell value
train.iloc[1]['premise']
train.iloc[1]['hypothesis']

In [19]:
dataset = nlp.Dataset.from_pandas(train_filtered)
ds_splitted = dataset.train_test_split(test_size=0.1)

In [24]:
ds_splitted['train']['premise'][1]

"because i always had to do it and so i just pay someone else to do it and they do the they do the cutting they fertilize they um edge and um i think this year i'm going to have some landscaping put in"

# Model

In [9]:
%%capture
!pip install pytorch_lightning
!pip install transformers
!pip install nlp

In [1]:
import torch as th
import pytorch_lightning as pl
import nlp
import transformers

In [5]:
#example of tokenizing 2 sentences into
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
sequence_a = "HuggingFace is based in NYC"
sequence_b = "Where is HuggingFace based?"
encoded_dict = tokenizer(sequence_a, sequence_b,                     
                         max_length=32, 
                         truncation = True,
                         pad_to_max_length=True); encoded_dict
#decoded = tokenizer.decode(encoded_dict["input_ids"])

{'input_ids': [101, 20164, 10932, 2271, 7954, 1110, 1359, 1107, 17520, 102, 2777, 1110, 20164, 10932, 2271, 7954, 1359, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [ ]:
from nlp import load_dataset
datasets = load_dataset('imdb')
print(datasets)

In [ ]:
class Sentences(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.brt = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.layer1 = torch.nn.Linear(64,32)
        self.layer2 = torch.nn.Linear(32,3)
        self.loss = th.nn.CrossEntropyLoss(reduction='none')

    def prepare_data(self):
        tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

        def _tokenize(x):
            return tokenizer(
                    x['text'], 
                    max_length=20, 
                    truncation = True,
                    pad_to_max_length=True)
            
        def _prepare_ds():
            #needs to be re-written for reading data from pandas
            df = pd.read_csv('train.csv')
            df_filtered  = df.filter(['premise','hypothesis','label'])
            dataset = Dataset.from_pandas(df_filtered)
            ds_flt = dataset.train_test_split(test_size=0.1)
            #ds_train, ds_test = nlp.load_dataset('imdb', split=['train[:500]', 'test[:20]'])
            ds_flt['train'] = ds_flt['train'].map(_tokenize, batched=True)
            ds_flt['train'].set_format(type='torch',columns = ['input_ids','token_type_ids','label','attention_mask'])
            ds_flt['test'] = ds_flt['test'].map(_tokenize, batched=True)
            ds_flt['test'].set_format(type='torch',columns = ['input_ids','token_type_ids','label','attention_mask'])
            #print('ds_train type - {}'.format(type(ds_flt['train'])))
            #print('ds_test type - {}'.format(type(ds_flt['test'])))

            return ds_train, ds_test

        self.train_ds, self.test_ds = _prepare_ds()      

    def forward(self, input_ids, masks):

        logits, = self.brt(input_ids, masks)
        out = self.layer1(logits)
        out = th.nn.reLU(out)
        out = self.layer2(out)
        #print ('logits- {} -- other stuff'.format(logits))
        return logits

    def training_step(self, batch, batch_idx):
        #print('training step function')
        logits = self.forward(batch['input_ids'],batch['attention_mask'], batch['token_type_ids'])
        loss = self.loss(logits, batch['label']).mean()

        return {'loss': loss, 'log': {'train_loss': loss}}

    def validation_step(self, batch, batch_idx):
        #print('validation step function')
        logits = self.forward(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
        loss = self.loss(logits, batch['label'])
        acc = (logits.argmax(-1) == batch['label']).float()

        return {'loss': loss, 'acc': acc}

    def validation_epoch_end(self, outputs):
        loss = th.cat([o['loss'] for o in outputs], 0).mean()
        acc = th.cat([o['acc'] for o in outputs], 0).mean()
        out = {'val_loss': loss, 'val_acc': acc}
        return {**out, 'log': out}

    def train_dataloader(self):
        #print('train_dataloader')
        return th.utils.data.DataLoader(
                self.train_ds,
                batch_size=1,
                drop_last=True,
                shuffle=True,
                )

    def val_dataloader(self):
        #print('val_dataloader')
        return th.utils.data.DataLoader(
                self.test_ds,
                batch_size=32,
                drop_last=False,
                shuffle=True,
                )

    def configure_optimizers(self):
        return th.optim.SGD(
            self.parameters(),
            lr=0.01,
            momentum=0.9,
        )

In [ ]:

def model_fit():
    model = Sentences()
    trainer = pl.Trainer(
        #default_root_dir='logs',
        gpus=(1 if th.cuda.is_available() else 0),
        max_epochs=4,
        fast_dev_run=False,
        logger=pl.loggers.TensorBoardLogger('logs/', name='imdb'),
    )
    trainer.fit(model)

!rm -rf ./logs/

model_fit()

# Sandbox